In [19]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import math
print(torch.cuda.is_available())

True


In [20]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [22]:
static = pd.read_csv('static-pre-process.csv')

In [23]:
from sklearn.preprocessing import MinMaxScaler
static_1 = static.copy()
scaler = MinMaxScaler(feature_range=(0, 1))
static_1['race_encoded'] = scaler.fit_transform(static_1[['race_encoded']])
static_1['icu_outtime'] = pd.to_datetime(static_1['icu_outtime'])
static_1['icu_intime'] = pd.to_datetime(static_1['icu_intime'])

# Calculate ICU stay duration in hours and keep only the total number of hours
static_1['icu_hours'] = (static_1['icu_outtime'] - static_1['icu_intime']).dt.total_seconds() / 3600
static_1.drop(['admission_type','first_careunit'], axis=1, inplace=True)
static_1.sort_values(by='id', ascending=True, inplace=True)

In [24]:
static_2 = static_1.loc[:,'admission_age':'gender_encoded']
numpy_array = static_2.to_numpy()
static_data = torch.tensor(numpy_array, dtype=torch.float32)
static_data.shape

torch.Size([20414, 16])

In [25]:
ts = pd.read_csv('dynamic-pre-process.csv')
ts1 = ts.copy()

In [26]:
grouped = ts1.groupby('id')
tensor_list = []

for name, group in grouped:
    values = group.loc[:,'albumin':].values
    tensor_list.append(torch.tensor(values, dtype=torch.float))

In [27]:
lst_ts = [t.shape[0] for t in tensor_list]
average_length = sum(lst_ts) / len(lst_ts)
math.ceil(average_length)

5

In [28]:
ave_length = 5
num_features = len(ts1.columns) - list(ts1.columns).index('albumin')
final_tensors = []
for t in tensor_list:
    if t.shape[0] > ave_length:
        t = t[:ave_length, :]
    elif t.shape[0] < ave_length:
        padding_needed = ave_length - t.shape[0]
        padding = torch.zeros(padding_needed, num_features, dtype=t.dtype)
        t = torch.cat([t, padding], dim=0)
    final_tensors.append(t)

In [29]:
ts_data = torch.stack(final_tensors)
ts_data.shape

torch.Size([20414, 5, 68])

In [30]:
label_1 = static_1['icu_death']
labels = torch.tensor(label_1.values.reshape(-1, 1), dtype=torch.float32)
labels.shape

torch.Size([20414, 1])

In [31]:
text = pd.read_csv('cleaned_notes.csv')
text = text[['id','text']]
text['text'] = text['text'].str.replace('\n', ' ', regex=False)
text.sort_values(by='id', ascending=True, inplace=True)

In [32]:
text_merged = text.groupby('id')['text'].agg(' '.join).reset_index()
text_data = text_merged['text'].tolist()

In [33]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, static_data, ts_data, texts, labels):
        self.static_data = static_data
        self.ts_data = ts_data
        self.texts = texts
        self.labels = labels
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        text_tokens = self.tokenizer(self.texts[idx], return_tensors='pt', padding=True, truncation=True, max_length=128, add_special_tokens=True)
        return (
            self.static_data[idx],
            self.ts_data[idx],
            text_tokens['input_ids'].squeeze(0),
            text_tokens['attention_mask'].squeeze(0),
            self.labels[idx]
        )

In [34]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    static_data, ts_data, input_ids, attention_masks, labels = zip(*batch)

    input_ids = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_masks = pad_sequence(attention_masks, batch_first=True, padding_value=0)

    static_data = torch.stack(static_data)
    ts_data = torch.stack(ts_data)
    labels = torch.tensor(labels)

    return static_data, ts_data, input_ids, attention_masks, labels

In [35]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

(train_static, test_static, 
 train_ts, test_ts, 
 train_texts, test_texts, 
 train_labels, test_labels) = train_test_split(static_data, ts_data, text_data, labels, test_size=0.2, random_state=42)

train_dataset = CustomDataset(train_static, train_ts, train_texts, train_labels)
test_dataset = CustomDataset(test_static, test_ts, test_texts, test_labels)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

In [36]:
class MultiInputModel(nn.Module):
    def __init__(self):
        super(MultiInputModel, self).__init__()
        self.static_layer = nn.Sequential(
            nn.Linear(16, 32), # 16 features
            nn.ReLU()
        )
        self.lstm = nn.LSTM(input_size=68, hidden_size=32, batch_first=True) #68 features
        self.bert = bert_model
        self.fc = nn.Sequential(
            nn.Linear(32 + 32 + 768, 64),  # LSTM and static 32 dim，BERT output 768 dim
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )
    
    def forward(self, static_data, ts_data, input_ids, attention_mask):
        static_features = self.static_layer(static_data)
        _, (hidden, _) = self.lstm(ts_data)
        lstm_features = hidden[-1]
        text_features = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
        combined_features = torch.cat([static_features, lstm_features, text_features], dim=1)
        output = self.fc(combined_features)
        return output

In [37]:
torch.cuda.empty_cache()

In [41]:
from sklearn.metrics import roc_auc_score
for i in [(8,0.0001),(16,0.0001),(32,0.0001),(8,0.001),(16,0.001),(32,0.001)]:
    class MultiInputModel(nn.Module):
        def __init__(self):
            super(MultiInputModel, self).__init__()
            self.static_layer = nn.Sequential(
                nn.Linear(16, i[0]), # 16 features
                nn.ReLU()
            )
            self.lstm = nn.LSTM(input_size=68, hidden_size=i[0], batch_first=True) #68 features
            self.bert = bert_model
            self.fc = nn.Sequential(
                nn.Linear(i[0] + i[0] + 768, i[0]),  # LSTM and static 32 dim，BERT output 768 dim
                nn.ReLU(),
                nn.Linear(i[0], 1),
                nn.Sigmoid()
            )
        
        def forward(self, static_data, ts_data, input_ids, attention_mask):
            static_features = self.static_layer(static_data)
            _, (hidden, _) = self.lstm(ts_data)
            lstm_features = hidden[-1]
            text_features = self.bert(input_ids=input_ids, attention_mask=attention_mask).pooler_output
            combined_features = torch.cat([static_features, lstm_features, text_features], dim=1)
            output = self.fc(combined_features)
            return output


    torch.cuda.empty_cache()
    model = MultiInputModel()

    loss_function = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=i[1])
    num_epochs = 10

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    for epoch in range(num_epochs):
        for data in train_loader:
            static_data, ts_data, input_ids, attention_mask, labels = data

            static_data = static_data.to(device)
            ts_data = ts_data.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device).float() 

            output = model(static_data, ts_data, input_ids, attention_mask)
            labels = labels.unsqueeze(1)
            loss = loss_function(output, labels.float())
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch+1}, Loss: {loss.item()}',i)
    model.eval()

    total_correct = 0
    num_samples = 0

    with torch.no_grad():
        for data in test_loader:
            static_data, ts_data, input_ids, attention_mask, labels = data

            static_data = static_data.to(device)
            ts_data = ts_data.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device).float()

            output = model(static_data, ts_data, input_ids, attention_mask)
            labels = labels.unsqueeze(1)
            predictions = (output > 0.5).float()
            correct = (predictions == labels).float().sum()

            total_correct += correct
            num_samples += labels.size(0)

    accuracy = total_correct / num_samples
    print(f'Test Accuracy: {accuracy.item()}',i)

    model.eval()

    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for data in test_loader:
            static_data, ts_data, input_ids, attention_mask, labels = data

            static_data = static_data.to(device)
            ts_data = ts_data.to(device)
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device).float()

            output = model(static_data, ts_data, input_ids, attention_mask)

            all_predictions.extend(output.view(-1).cpu().numpy())
            all_labels.extend(labels.view(-1).cpu().numpy())

    auc = roc_auc_score(all_labels, all_predictions)
    print(f'Test AUC: {auc}',i)
    

Epoch 1, Loss: 0.30614933371543884 (8, 0.0001)
Epoch 2, Loss: 0.5118962526321411 (8, 0.0001)
Epoch 3, Loss: 0.3002335727214813 (8, 0.0001)
Epoch 4, Loss: 0.6778562068939209 (8, 0.0001)
Epoch 5, Loss: 0.3189200460910797 (8, 0.0001)
Epoch 6, Loss: 0.2772770822048187 (8, 0.0001)
Epoch 7, Loss: 0.10982435941696167 (8, 0.0001)
Epoch 8, Loss: 0.2964090406894684 (8, 0.0001)
Epoch 9, Loss: 0.1279366910457611 (8, 0.0001)
Epoch 10, Loss: 0.10461699217557907 (8, 0.0001)
Test Accuracy: 0.8956649899482727 (8, 0.0001)
Test AUC: 0.6642409373752312 (8, 0.0001)
Epoch 1, Loss: 0.09966601431369781 (16, 0.0001)
Epoch 2, Loss: 0.3088759481906891 (16, 0.0001)
Epoch 3, Loss: 0.6870419979095459 (16, 0.0001)
Epoch 4, Loss: 0.2912318706512451 (16, 0.0001)
Epoch 5, Loss: 0.2893545925617218 (16, 0.0001)
Epoch 6, Loss: 0.529589056968689 (16, 0.0001)
Epoch 7, Loss: 0.3206197917461395 (16, 0.0001)
Epoch 8, Loss: 0.14706763625144958 (16, 0.0001)
Epoch 9, Loss: 0.08707202970981598 (16, 0.0001)
Epoch 10, Loss: 0.525754

In [21]:
model.eval()

total_correct = 0
num_samples = 0

with torch.no_grad():
    for data in test_loader:
        static_data, ts_data, input_ids, attention_mask, labels = data

        static_data = static_data.to(device)
        ts_data = ts_data.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device).float()

        output = model(static_data, ts_data, input_ids, attention_mask)
        labels = labels.unsqueeze(1)
        predictions = (output > 0.5).float()
        correct = (predictions == labels).float().sum()

        total_correct += correct
        num_samples += labels.size(0)

accuracy = total_correct / num_samples
print(f'Test Accuracy: {accuracy.item()}')


Test Accuracy: 0.8802351355552673


In [38]:
from sklearn.metrics import roc_auc_score

model.eval()

all_predictions = []
all_labels = []

with torch.no_grad():
    for data in test_loader:
        static_data, ts_data, input_ids, attention_mask, labels = data

        static_data = static_data.to(device)
        ts_data = ts_data.to(device)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        labels = labels.to(device).float()

        output = model(static_data, ts_data, input_ids, attention_mask)

        all_predictions.extend(output.view(-1).cpu().numpy())
        all_labels.extend(labels.view(-1).cpu().numpy())

auc = roc_auc_score(all_labels, all_predictions)
print(f'Test AUC: {auc}')

NameError: name 'model' is not defined